# Базовые операции с Kafka

Кластер Apache Kafka доступен через:

In [ ]:
echo "$KAFKA_HOST":"$KAFKA_PORT"

## Создать топик

In [ ]:
kafka-topics --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --topic my-first-topic \
    --create \
    --partitions 1 \
    --replication-factor 1

## Получить список топиков

In [ ]:
kafka-topics --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --list

## Записать данные в топик

В топик будет записано десять сообщений 'Hello, World!':

In [ ]:
kafka-console-producer --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --topic my-first-topic \
    <<<$(yes 'Hello, World!' 2>/dev/null | head -n 10)

## Прочитать данные из топика

In [ ]:
kafka-console-consumer --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --topic my-first-topic \
    --from-beginning \
    --timeout-ms 10000

## Удалить топик

In [ ]:
kafka-topics --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --topic my-first-topic \
    --delete

In [ ]:
kafka-topics --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --list